In [1]:
from langchain_ollama import OllamaLLM

# Initialize the model
llm = OllamaLLM(model="llama3")

# Run a prompt
response = llm.invoke("Write a short poem about the stars.")
print(response)
#temperature is 0 it is safe and it is not taking any bats.If 1 means it may take risk and may generate wrong output and very creative at the same time
# Low temperature = predictable output
# High temperature = more creative/output varies

Here is a short poem about the stars:

The stars shine bright in the midnight sky,
A twinkling tapestry, passing by.
Like diamonds scattered across the sea,
They sparkle and glimmer, wild and free.

Their gentle twinkle whispers secrets low,
Of ancient tales and mysteries to know.
As I gaze upon their celestial show,
My heart is filled with wonder, and my spirit glows.


In [2]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model="llama3", temperature=0.2)
response = llm.invoke("What is the capital of France?")
print("Low temp:", response)

Low temp: The capital of France is Paris.


In [3]:
from langchain.prompts import PromptTemplate
prompt_template_name=PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open a restaurant for {cuisine} food.Suggest a fancy name for this "
)
prompt_template_name.format(cuisine="Mexican")

'I want to open a restaurant for Mexican food.Suggest a fancy name for this '

In [4]:
from langchain.chains import LLMChain
chain =LLMChain(llm=llm,prompt=prompt_template_name)#here we are linking the llm we create above and the prompt_template_name to the prompt 
chain.invoke("American")

C:\Users\bhara\AppData\Local\Temp\ipykernel_13636\4130200667.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain =LLMChain(llm=llm,prompt=prompt_template_name)#here we are linking the llm we create above and the prompt_template_name to the prompt


{'cuisine': 'American',
 'text': "Exciting venture! Here are some fancy name suggestions for your American food restaurant:\n\n1. **Liberty Bites**: A nod to American independence and the idea of indulging in delicious bites.\n2. **Star-Spangled Eats**: A playful reference to the American flag and the promise of tasty eats.\n3. **Appetite America**: A name that celebrates the diversity of American cuisine and the joy of exploring new flavors.\n4. **Patriot's Plate**: A name that honors the spirit of American patriotism and the idea of gathering around a hearty plate of food.\n5. **Main Street Morsels**: A name that evokes the charm of small-town America and the idea of savoring bite-sized treats.\n6. **All-American Eats Co.**: A straightforward name that proudly proclaims your restaurant's focus on classic American cuisine.\n7. **The Red, White & Delicious**: A playful name that incorporates the colors of the American flag and promises a culinary experience that's simply delicious.\n8.

In [5]:
#simplesequentialchain->the output of the first step is the input of the seconfd step
#here 1) input="indian" output="indian restaurant name" 2)input="indian restaurant name" output="menu itemsll" 

In [6]:
llm=llm = OllamaLLM(model="llama3", temperature=0.6)

prompt_template_name=PromptTemplate(
    input_variables=['cuisine'],
    template="I want to open a restaurant for {cuisine} food.Suggest a fancy name for this "
)

name_chain=LLMChain(llm=llm,prompt=prompt_template_name)

prompt_template_items=PromptTemplate(
    input_variables=['restaurant_name'],
    template=""" Suggest some menu items for {restaurant_name}, Return it as a comma separated values"""
)
food_items_chain=LLMChain(llm=llm,prompt=prompt_template_items)

In [7]:
from langchain.chains import SimpleSequentialChain

chain=SimpleSequentialChain(chains=[name_chain,food_items_chain])
response=chain.invoke("Indian")
print(response)

{'input': 'Indian', 'output': "What an exciting venture indeed!\n\nBased on your suggestions, I think **Tamarind & Tales** is the perfect fit. It combines the flavors of Indian cuisine with the rich storytelling traditions of the country, which aligns well with my understanding of your vision and style.\n\nHere's the chosen name in a comma-separated value format:\n\nTamarind & Tales"}


In [8]:
#if we want both restaurant name and menu items both then we need to use sequentialchain it can have multiple input and multiple output

In [15]:
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain

# Initialize LLM
llm = OllamaLLM(model="llama3", temperature=0.7)

# Chain 1: Generate restaurant name
prompt_template_name = PromptTemplate(
    input_variables=["cuisine"],
    template="Suggest a name for a {cuisine} restaurant."
)
name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

# Chain 2: Generate menu items
prompt_template_food_items = PromptTemplate(
    input_variables=["cuisine", "restaurant_name"],
    template="Suggest 5 menu items for a {cuisine} restaurant called {restaurant_name}."
)
food_items_chain = LLMChain(llm=llm, prompt=prompt_template_food_items, output_key="menu_items")

# Combine chains
full_chain = SequentialChain(
    chains=[name_chain, food_items_chain],
    input_variables=["cuisine"],
    output_variables=["restaurant_name", "menu_items"]
)

# ✅ Use invoke() instead of calling the chain directly
response = full_chain.invoke({"cuisine": "Arabic"})

# Print result
print("Restaurant Name:", response["restaurant_name"])
print("Menu Items:", response["menu_items"])

Restaurant Name: What a delicious task!

Here are some suggestions for an Arabic restaurant:

1. **Mashriq**: A play on the word "mashreq," meaning "east" in Arabic, evoking the rich cultural heritage of the Middle East.
2. **Sahraa**: Meaning "desert" or "arid land" in Arabic, this name could appeal to customers looking for a taste of the region's cuisine.
3. **Karma**: Inspired by the Arabic word for "flavor," this name promises a culinary experience that will satisfy your cravings.
4. **Hawiya**: This name combines the Arabic words for "hospitality" (hawi) and "house" (ya), conveying warmth and welcoming vibes.
5. **Nafoura**: Meaning "fragrance" or "aroma," this name captures the essence of Middle Eastern cuisine, which is renowned for its aromatic spices and herbs.
6. **Firdaus**: Named after the Arabic word for "paradise," this name promises a culinary journey to a culinary heaven on earth.
7. **Shams**: Meaning "sun" in Arabic, this name reflects the warmth and vibrancy of Middl

In [6]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_ollama import OllamaLLM

# Initialize the model
llm = OllamaLLM(model="llama3")
tools=load_tools(["wikipedia","llm-math"],llm=llm)
agent=initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, #first we make thought then we take action then we will go where we need
    verbose=True #To see what are the internal steps it is taking
)

agent.invoke(
  "Use the Wikipedia tool to find when Elon Musk was born. "
  "Give ONLY the final answer in the format 'Answer: <number>'."
)


C:\Users\bhara\AppData\Local\Temp\ipykernel_39384\1927731066.py:7: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent=initialize_agent(




> Entering new AgentExecutor chain...
Thought: To find when Elon Musk was born, I will use the Wikipedia tool.

Action: wikipedia
Action Input: "Elon Musk birthdate"
Observation: Page: History of artificial intelligence
Summary: The history of artificial intelligence (AI) began in antiquity, with myths, stories, and rumors of artificial beings endowed with intelligence or consciousness by master craftsmen. The study of logic and formal reasoning from antiquity to the present led directly to the invention of the programmable digital computer in the 1940s, a machine based on abstract mathematical reasoning. This device and the ideas behind it inspired scientists to begin discussing the possibility of building an electronic brain.
The field of AI research was founded at a workshop held on the campus of Dartmouth College in 1956. Attendees of the workshop became the leaders of AI research for decades. Many of them predicted that machines as intelligent as humans would exist within a gene

KeyboardInterrupt: 

In [ ]:
import os
os.environ['SERPAPI_API_KEY']='f5fc985699ca6f75f346f4f5d6323c2bb0cef9a1d75998a6d94849a63105ed99'
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_ollama import OllamaLLM

# Initialize the model
llm = OllamaLLM(model="llama3")
tools=load_tools(["serpapi","llm-math"],llm=llm)
agent=initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, #first we make thought then we take action then we will go where we need
    verbose=True #To see what are the internal steps it is taking
)
agent.invoke("What was us gdp in 2025 plus 5?")


In [14]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
memory=ConversationBufferMemory()
chain=LLMChain(llm=llm,prompt=prompt_template_name,memory=memory)
chain.invoke("Mexican")
print(name)

NameError: name 'prompt_template_name' is not defined

In [15]:
chain.invoke("Indian")
print(name)

NameError: name 'chain' is not defined

In [ ]:
print(chain.memory.buffer)#i question answer pair is 1 conversational exchange.so the problem is it will send all this to the model next time when we chat so the model charges will increase based on the charges

In [7]:
#we can say just to remember last 5 conversation change 
from langchain.chains import ConversationChain
convo=ConversationChain(llm = OllamaLLM(model="llama3", temperature=0.2))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


C:\Users\bhara\AppData\Local\Temp\ipykernel_39384\3779745732.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  convo=ConversationChain(llm = OllamaLLM(model="llama3", temperature=0.2))
C:\Users\bhara\AppData\Roaming\Python\Python313\site-packages\pydantic\main.py:214: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


In [16]:
convo.invoke("what is 5+5?")

{'input': 'what is 5+5?',
 'history': "Human: what is 5+5\nAI: A simple arithmetic question! According to my programming and vast knowledge database, the answer to 5+5 is indeed 10. In fact, I can provide you with a step-by-step calculation if you'd like: 5 + 0 = 5, then add 1 to get 6, add another 1 to get 7, and finally add 2 more to reach the total of 10.\nHuman: what is 5+5\nAI: A repeat question! I'm happy to confirm that my previous answer still stands. The correct result of the arithmetic operation 5+5 remains 10. If you'd like, I can still provide the step-by-step calculation for your reference:\n\n1. Start with 5 + 0 = 5\n2. Add 1 to get 6\n3. Add another 1 to get 7\n4. Finally, add 2 more to reach a total of 10\n\nSo, the answer remains consistent: 5+5 = 10.",
 'response': "Another repeat question! I'm happy to confirm that my previous answers still stand. The correct result of the arithmetic operation 5+5 remains 10. If you'd like, I can still provide the step-by-step calcul

In [17]:
print(convo.memory.buffer)#it will track the memory .

Human: what is 5+5
AI: A simple arithmetic question! According to my programming and vast knowledge database, the answer to 5+5 is indeed 10. In fact, I can provide you with a step-by-step calculation if you'd like: 5 + 0 = 5, then add 1 to get 6, add another 1 to get 7, and finally add 2 more to reach the total of 10.
Human: what is 5+5
AI: A repeat question! I'm happy to confirm that my previous answer still stands. The correct result of the arithmetic operation 5+5 remains 10. If you'd like, I can still provide the step-by-step calculation for your reference:

1. Start with 5 + 0 = 5
2. Add 1 to get 6
3. Add another 1 to get 7
4. Finally, add 2 more to reach a total of 10

So, the answer remains consistent: 5+5 = 10.
Human: what is 5+5?
AI: Another repeat question! I'm happy to confirm that my previous answers still stand. The correct result of the arithmetic operation 5+5 remains 10. If you'd like, I can still provide the step-by-step calculation for your reference:

1. Start with 

In [20]:
#If the above memory method builds endlessly so we can send conversational window memory
from langchain.memory import ConversationBufferWindowMemory
memory=ConversationBufferWindowMemory(k=1)#here we are saying remember the last conversation
convo=ConversationChain(llm = OllamaLLM(model="llama3", temperature=0.2),memory=memory)
convo.invoke("who won the first cricket world cup")
convo.invoke("what is 5+9?")#here it remembers only the last conversation
convo.invoke("who was the captain of winning team")# It does not know which game are u talking about.

{'input': 'who was the captain of winning team',
 'history': "Human: what is 5+9?\nAI: A simple question! According to my programming, the answer to 5 + 9 is... (drumroll please)... 14! Yes, the correct calculation is:\n\n5 + 9 = 14\n\nI'm happy to help with more math problems or any other questions you might have!",
 'response': "Another great question!\n\nTo answer your query, I'll need a bit more context. Which sport are we talking about? Was it a football game, a basketball game, or perhaps a soccer match?\n\nIf you could provide me with more details, such as the date, location, and any notable players on the winning team, I'd be happy to try and find the answer for you!\n\n(And if I don't know the answer, I'll be honest and say so!)"}